## Open Sourced Under Version 2 Of The Apache License

In [ ]:
from collections import defaultdict, deque
from random import randint, random
from operator import itemgetter
from uuid import uuid4
from math import floor

## Monte-Carlo Simulation Of Amulet Platform's Reward Function

Uses market experts with a high probability of a correct response, experts which guess randomly every time, and experts with a low probability of a correct response. Also uses random stakes, and randomizes choice of answers for both the expert types who have high probability of success and for the expert types who have low probability of success.

Authored By Horia Margarit On 2018-04-20

### Simple And Exponential Moving Averages Both Extend The Same Abstract Class

In [ ]:
class MovingAverage(object):
    def __init__(self, *args, **kwargs):
        self._curr = 0.0
        super(MovingAverage, self).__init__()
    
    def __call__(self, *args, **kwargs):
        prev = self._curr
        self._update_curr(*args, **kwargs)
        return prev, self._curr
    
    def _update_curr(self, *args, **kwargs):
        raise NotImplementedError('MovingAverage abstract class cannot implement specific algorithm!')

In [ ]:
class SMA(MovingAverage):
    def __init__(self, *args, **kwargs):
        m = int(args[0])
        self._n = float(m)
        self._vals = deque([0.0]*m)
        super(SMA, self).__init__(*args, **kwargs)
    
    def _update_curr(self, *args, **kwargs):
        _ = self._vals.popleft()
        self._vals.append(float(args[0]))
        self._curr = sum(self._vals) / self._n

In [ ]:
class EMA(MovingAverage):
    def __init__(self, *args, **kwargs):
        self._a = float(args[0])
        self._b = 1.0 - self._a
        super(EMA, self).__init__(*args, **kwargs)
    
    def _update_curr(self, *args, **kwargs):
        self._curr *= self._b
        self._curr += self._a * float(args[0])

### Reputation Manager Is A Quasi-meta Class Which Bookkeeps The Reputation Of Individual Experts

In [ ]:
class ReputationManager(object):
    def __init__(self, *args, **kwargs):
        window = int(kwargs.pop('window', 5))
        alpha = float(kwargs.pop('alpha', 0.2))
        if bool(kwargs.pop('ema', False)):
            self._rhos = defaultdict(lambda: EMA(alpha))
        else:
            self._rhos = defaultdict(lambda: SMA(window))
        super(ReputationManager, self).__init__()
    
    def __call__(self, *args, **kwargs):
        e_id, correct, value = args
        prev, _ = self._rhos[e_id](correct)
        return prev

### Requests For Proposal Are Constructed By The RFP Builder For Quick Iterative Use

RFP Builder takes as input the max entires, the max value, and the class for the reputation management, as well as parameters for said class. It instatiates to a callable object. Upon each call, said object constructs and returns a new RFP.

In [ ]:
class RfpBuilder(object):
    def __init__(self, *args, **kwargs):
        self._max_entries = int(args[0])
        self._max_value   = int(args[1])
        self._oracle      = args[2](**kwargs)
        super(RfpBuilder, self).__init__()
    
    def __call__(self, *args, **kwargs):
        n = randint(2, self._max_entries)
        v = randint(1, self._max_value)
        a = int(floor(n / 2.0))
        return RFP(self._oracle, n, v, a)

### Requests For Proposal Are Implemented As An Object Which Also Computes Alphas Of Reward Function

RFP is constructed only by RFP Builder and results in a callable object. Upon calling said object, it stores necessary information to compute alphas. Caveat: all calls that will ever occur must complete **before** calling the method *compute_alphas*. Otherwise the computation will be meaningless and wrong.

In [ ]:
class RFP(object):
    def __init__(self, *args, **kwargs):
        self._phis = defaultdict(lambda: 0.0)
        self._n, self._v, self._a = map(int, args[1:4])
        self._prob_rand_guess_correct = 1.0 / self._n
        self._oracle = args[0] if isinstance(args[0], ReputationManager) else None
        super(RFP, self).__init__()
    
    def __call__(self, *args, **kwargs):
        e_id, ans = args
        if ans >= 0: # expert posted stake and responded
            correct = ans == self._a
            rho = self._oracle(e_id, correct, self._v)
        else:
            correct = False
            rho = 0.0
        if correct:
            self._phis[e_id] = rho - self._prob_rand_guess_correct
        else:
            self._phis[e_id] = 0.0
    
    @property
    def num_entries(self):
        return self._n
    
    @property
    def value(self):
        return self._v
    
    def compute_alphas(self, *args, **kwargs):
        rs = 0.0
        for k, v in self._phis.items():
            self._phis[k] = max(0.0, v)
            rs += self._phis[k]
        alphas = defaultdict(lambda: 0.0)
        for k, v in self._phis.items():
            alphas[k] = v / rs if rs != 0.0 else 0.0
            self._phis[k] = 0.0
        return sorted(alphas.items(), key=itemgetter(0))

### Randomly Guessing Experts And Low And High Performing Experts Extend The Expert Abstract Class

They all extend the Expert class, which instantiates to a callable object. Said object is not called with an RFP object, as that would reveal the answer and enable the expert to change the alphas for the reward function. Instead it takes three arguments: the number of choices for the RFP, the value of the RFP (this is the amount bid by investors), and the stake required of the expert to pay to respond.

All expert types must be instatiated with payout odds and with a personal budget. Payout odds determine the maximum stake an expert is willing to post as the ratio of the value of the RFP divided by the payout odds. Eg., investor bid for an RFP of 14 with payout odds of 7 means that the expert is willing to post a stake up to but no more than 2.

In [ ]:
class Expert(object):
    def __init__(self, *args, **kwargs):
        self._payout_odds = float(args[0])
        self._personal_budget = float(args[1])
        self._id = int(kwargs.pop('id', uuid4().int))
        super(Expert, self).__init__()
    
    def __call__(self, *args, **kwargs):
        n, v, s = args
        if self._will_respond(v, s):
            self._personal_budget -= s
            a = self._my_response(n, v, s)
        else:
            a = -1
        return self._id, a
    
    def _will_respond(self, *args, **kwargs):
        v, s = args
        can_respond  = s <= self._personal_budget
        can_respond &= s <= v / self._payout_odds
        return can_respond
    
    def _my_response(self, *args, **kwargs):
        raise NotImplementedError('Expert abstract class cannot implement any response strategies!')

In [ ]:
class HighPerformanceExpert(Expert):
    def __init__(self, *args, **kwargs):
        self._eps = 1e-3
        super(HighPerformanceExpert, self).__init__(*args, **kwargs)
    
    def _my_response(self, *args, **kwargs):
        n, v, s = args
        p_wrong = 1.0 / n - self._eps
        p_right = 1.0 - p_wrong
        a = int(floor(n / 2.0))
        r = random()
        if r < p_right:
            # with probability p_right, return a
            ans = a
        else:
            # with probability p_wrong, choose either the answer before or after uniformly at random
            split = p_wrong / 2
            ans = a-1 if r < (p_right + split) else a+1
        return ans

In [ ]:
class RandomChoiceExpert(Expert):
    def __init__(self, *args, **kwargs):
        super(RandomChoiceExpert, self).__init__(*args, **kwargs)
    
    def _my_response(self, *args, **kwargs):
        n, v, s = args
        ans = randint(0, n)
        return ans

In [ ]:
class LowPerformanceExpert(Expert):
    def __init__(self, *args, **kwargs):
        self._eps = 1e-3
        super(LowPerformanceExpert, self).__init__(*args, **kwargs)
    
    def _my_response(self, *args, **kwargs):
        n, v, s = args
        p_right = 1.0 / n - self._eps
        p_wrong = 1.0 - p_right
        a = int(floor(n / 2.0))
        r = random()
        if r < p_right:
            # with probability p_right, return a
            ans = a
        else:
            # with probability p_wrong, choose either the answer before or after uniformly at random
            split = p_wrong / 2
            ans = a-1 if r < (p_right + split) else a+1
        return ans

### Proctor Collects The Input To The Payout Pot For Each RFP Then Asks The Experts If They Wish To Respond

In [ ]:
# build RFPs with a max choice number of 20 and a max value of 1000
# using a reputation manager which tracks reputation by simple moving average of window size 10
rfp_builder = RfpBuilder(20, 1000, ReputationManager, ema=False, window=60)
# rfp_builder = RfpBuilder(20, 1000, ReputationManager, ema=True, alpha=0.05)

# create 101 requests for prediction with a random stake from 2 to 11
n_rfps = 101
rfp_s = tuple(rfp_builder() for _ in range(n_rfps))
stakes = tuple(randint(2, 11) for _ in range(n_rfps))

# initialize 1000 random choice experts with payout odds of 7 and with initial budgets of 100
num_exp = 1000
payout_odds = [7]*num_exp
expert_budgets = [100]*num_exp
rand_experts = dict((j+1, RandomChoiceExpert(payout_odds[j], expert_budgets[j], id=j+1))
                    for j in range(num_exp))

# initialize 1000 high performance experts with same payout odds and same initial budgets as random choice experts
high_experts = dict((num_exp+j+1, HighPerformanceExpert(payout_odds[j], expert_budgets[j], id=num_exp+j+1))
                    for j in range(num_exp))

# initialize 1000 high performance experts with same payout odds and same initial budgets as random choice experts
low_experts = dict((2*num_exp+j+1, LowPerformanceExpert(payout_odds[j], expert_budgets[j], id=2*num_exp+j+1))
                   for j in range(num_exp))

# separately store the pot sizes so they can be incremented by posted stakes of responding experts
pot_sizes = list(rfp.value for rfp in rfp_s)

# store alphas / payout ratios and store expected budgets after simulating responses to each RFP
payout_ratios, expected_budgets = list(), ([], [], [])
num_exp = float(num_exp)

# simulation run: for each RFP, consecutively ask the experts to respond to the prediction request
for j, rfp in enumerate(rfp_s):
    n, v, s = rfp.num_entries, rfp.value, stakes[j]
    for expert in high_experts.values():
        e_id, a = expert(n, v, s)
        rfp(e_id, a)
        if a >= 0: # expert responded with something so increment pot by posted stake
            pot_sizes[j] += s
    for expert in rand_experts.values():
        e_id, a = expert(n, v, s)
        rfp(e_id, a)
        if a >= 0: # expert responded with something so increment pot by posted stake
            pot_sizes[j] += s
    for expert in low_experts.values():
        e_id, a = expert(n, v, s)
        rfp(e_id, a)
        if a >= 0: # expert responded with something so increment pot by posted stake
            pot_sizes[j] += s
    # only compute payout ratios / alphas after all experts have responded to a particular RFP
    payout_ratios.append(rfp.compute_alphas())
    # update budget of all experts after RFP has been responded to by everyone
    for e_id, alpha in payout_ratios[-1]:
        if e_id in high_experts:
            high_experts[e_id]._personal_budget += alpha * pot_sizes[j]
        elif e_id in low_experts:
            low_experts[e_id]._personal_budget += alpha * pot_sizes[j]
        else:
            rand_experts[e_id]._personal_budget += alpha * pot_sizes[j]
    # after each RFP of random guessting, print the estimated expected budget of random choice experts
    rs = sum(expert._personal_budget for expert in high_experts.values())
    expected_budgets[0].append(rs / num_exp)
    rs = sum(expert._personal_budget for expert in rand_experts.values())
    expected_budgets[1].append(rs / num_exp)
    rs = sum(expert._personal_budget for expert in low_experts.values())
    expected_budgets[2].append(rs / num_exp)

# for now, avoid plots, and instead print the expected budgets
for h, r, l in zip(*expected_budgets):
    print("high probability expert: %.6f\trandom guessing expert: %.6f\tlow probability expert: %.6f" % (h, r, l))